In [1]:
import numpy as np
import json
import cv2
import os

In [18]:
# Prepare object points for 30mm grid
objp = np.zeros((5*7,3), np.float32)
# objp[:,:2] = np.mgrid[0:7,0:5].T.reshape(-1,2)

objp[:,:2] = np.mgrid[0:5,0:7].T.reshape(-1,2)

# objp

In [19]:
image_folder = "./calibration_data/visible/"
point_folder = "./extracted_points/visible/"

# image_folder = "./calibration_data/thermal/"
# point_folder = "./extracted_points/thermal/"

# image_folder = "./calibration_data/thermal_dots_binary/"
# point_folder = "./extracted_points/thermal_dots/"

objpoints = []
imgpoints = []
image_size = None

images = os.listdir(image_folder)
for fname in images:
	# Generate image points
	gray = cv2.imread(os.path.join(image_folder, fname), cv2.IMREAD_GRAYSCALE)

	# Set blob detector parameters
	params = cv2.SimpleBlobDetector_Params()
	params.filterByCircularity = False
	params.filterByConvexity = False
	params.filterByInertia = False
	params.filterByArea = True
	params.minArea = 4

	# Create a detector with the parameters
	ver = (cv2.__version__).split('.')
	if int(ver[0]) < 3 : detector = cv2.SimpleBlobDetector(params)
	else : detector = cv2.SimpleBlobDetector_create(params)

	# Find Corners
	ret = False
	# ret, corners = cv2.findCirclesGrid(gray, (7, 5), None, blobDetector=detector)
	ret, corners = cv2.findChessboardCorners(gray, (5,7), None)
	print(ret)
	
	# Draw corners
	out = cv2.drawChessboardCorners(cv2.cvtColor(gray, cv2.COLOR_GRAY2BGR), (7, 5), corners, ret)
	cv2.namedWindow("img", cv2.WINDOW_NORMAL)
	cv2.imshow("img", out)
	while cv2.waitKey(0) != ord('q'): continue
	cv2.destroyAllWindows()

	# Save image points
	if ret:
		points = corners
		criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
		points = cv2.cornerSubPix(gray, corners, (11,11), (-1,-1), criteria)

	else:
		name = fname.split(".")[0]
		points = np.loadtxt(os.path.join(point_folder, f"{name}.csv"), delimiter=",", dtype=float)
		points = np.array([np.array(p) for p in points]).astype('float32')

	imgpoints.append(points)

	# Save object points
	objpoints.append(objp)

True
True
True
True
True
True
True
True
True


In [27]:
# Compute parameters
h, w = gray.shape[:2]
vals = cv2.calibrateCamera(objpoints, imgpoints, (w,h), None, None)
keys = ("ret", "mtx", "dist", "rvecs", "tvecs")

params = dict(zip(keys, vals))
del params["rvecs"]
del params["tvecs"]

In [28]:
# Compute new camera matrix
vals = cv2.getOptimalNewCameraMatrix(params["mtx"], params["dist"], (w,h), 1, (w,h))
keys = ("newcameramtx", "roi")

params.update(dict(zip(keys, vals)))
params

{'ret': 0.18752077616544316,
 'mtx': array([[524.36570593,   0.        , 322.57309627],
        [  0.        , 525.87381374, 242.91852868],
        [  0.        ,   0.        ,   1.        ]]),
 'dist': array([[ 0.21235806, -0.47267247, -0.00106177, -0.00123414,  0.28311425]]),
 'newcameramtx': array([[534.82815439,   0.        , 321.82941307],
        [  0.        , 536.09892081, 242.44036121],
        [  0.        ,   0.        ,   1.        ]]),
 'roi': (3, 2, 634, 474)}

In [29]:
# Test Undistort
cv2.namedWindow("img", cv2.WINDOW_NORMAL)

images = os.listdir(image_folder)
for fname in images:
	# Load image
	img = cv2.imread(os.path.join(image_folder, fname), cv2.IMREAD_UNCHANGED)

	# Undistort
	img = cv2.undistort(img, params["mtx"], params["dist"], None, params["newcameramtx"])

	# Crop the image
	x, y, w, h = params["roi"]
	img = img[y:y+h, x:x+w]

	# Show image
	cv2.imshow("img", img)
	while cv2.waitKey(0) != ord('q'): continue

cv2.destroyAllWindows()

In [30]:
# Check if object is iterable
def is_iter(obj):
	try: iter(obj)
	except: return False
	return True

# Convert iterables to lists
def recursive_convert(some_iter):
	# Convert parent elements
	if type(some_iter) == np.ndarray:
		some_iter = some_iter.tolist()
	else: some_iter = list(some_iter)

	# Convert child elements
	for idx, elem in enumerate(some_iter):
		if is_iter(elem):
			some_iter[idx] = recursive_convert(elem)
	
	return some_iter

# Convert numpy arrays to python arrays
for k in params:
	if is_iter(params[k]):
		params[k] = recursive_convert(params[k])

print(params)

# Save calibration parameters to JSON
with open("visible_params.json", "w") as f:
	json.dump(params, f, indent="\t")
	f.close()

{'ret': 0.18752077616544316, 'mtx': [[524.365705929502, 0.0, 322.57309626948467], [0.0, 525.8738137402916, 242.91852868340916], [0.0, 0.0, 1.0]], 'dist': [[0.21235806220663594, -0.4726724653162688, -0.0010617699345519614, -0.0012341367248378263, 0.2831142450597614]], 'newcameramtx': [[534.828154385763, 0.0, 321.8294130720405], [0.0, 536.0989208132096, 242.4403612135718], [0.0, 0.0, 1.0]], 'roi': [3, 2, 634, 474]}
